In [ ]:
%conda install mlflow
%conda install pytest
%conda install tensorflow

In [ ]:
import mlflow, time, pickle, sys
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
#from tensorflow.keras.applications import VGG16

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

from sklearn.metrics import f1_score  # Import f1_score

sys.path.append('/home/w4z3/Git/MLOPS-cifar10')

from src import TRAIN_DATA_DIR, TEST_DATA_DIR, MODELS_DIR


In [ ]:
# Define the desired input size
input_size = (75, 75)

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Apply InceptionV3 preprocessing
    zoom_range=0.2,
    horizontal_flip=True
)

batch_size = 32  # Adjust as needed

train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [4]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
num_epochs = 10  # Adjust as needed
batch_size = 32  # Adjust as needed

In [ ]:
mlflow.set_experiment("cifar10")
mlflow.sklearn.autolog(log_model_signatures=False, log_datasets=False)

with mlflow.start_run():
  mlflow.log_params({
    "model_name": "imagenet-cifar10" ,
    "dataset_name": "CIFAR-10 Test" ,
    "num_epochs": num_epochs ,
    "batch_size": batch_size
  })

  model.fit(train_generator, epochs=num_epochs, batch_size=batch_size)
  model.save(MODELS_DIR/'cifar10.h5')
  #loaded_model = tf.keras.models.load_model('your_model.h5')

  evaluation = model.evaluate(test_generator)

  loss = evaluation[0]
  accuracy = evaluation[1]
  mlflow.log_metric("evaluation_loss", loss)
  mlflow.log_metric("evaluation_accuracy", accuracy)

  # Calculate F1 score
  y_true = test_generator.classes
  y_pred = model.predict(test_generator)
  y_pred = y_pred.argmax(axis=-1)  # Convert predicted probabilities to class labels
  f1score = f1_score(y_true, y_pred, average='weighted')
  mlflow.log_metric("evaluation_f1score", f1score)

  # Log model
  with open(MODELS_DIR / "cifar10.pkl", "wb") as pickle_file:
      pickle.dump(model, pickle_file)
  